<a href="https://colab.research.google.com/github/Alexandru-AndreiOana/CVClassification/blob/master/Labs_1and2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Idei
- lungimea frazelor, cuvintelor
- punctuatie
- vocabular - regionalisme, neologisme, adjective, verbe, prepozitii, nume proprii (indica citari, entitati), pronume, interjectii, conjuctii, marcartori discursivi, cuvinte de legatura;
- frecventele cuvintelor (cum comparăm două texte de lungimi diferite din perspectiva frecvențelor?)
- expresii, figuri de stil,  
- timpul verbelor,
- logica de argumentare (de la general la specific, bottom-up); structura textului
- nr de cuvinte / idee
- emotii, cat de descrpitiv e textul, dinamica textului (concis), relatare la pers I sau obiectiv

## Introducere spaCy

In [ ]:
! pip install -U spacy

In [ ]:
! python -m spacy download en_core_web_sm

2023-10-25 11:31:27.478526: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 11:31:27.478622: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 11:31:27.478729: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-25 11:31:29.538491: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy

NLP = spacy.load('en_core_web_sm')


In [ ]:
type(NLP)

spacy.lang.en.English

In [ ]:
text = "Me and John Doe went to the University of Bucharest. We were very happy about that."

In [ ]:
doc = NLP(text)

In [ ]:
doc

Me and John Doe went to the University of Bucharest. We were very happy about that.

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
for entity in doc.ents:
    print(entity, ' ', entity.label_)

John Doe   PERSON
the University of Bucharest   ORG


In [ ]:
list(doc.noun_chunks)

[Me, John Doe, the University, Bucharest, We, that]

In [ ]:
nc = list(doc.noun_chunks)
chunk = nc[2]
print(chunk)

the University


In [ ]:
type(chunk)

spacy.tokens.span.Span

In [ ]:
type(doc[2:8])

spacy.tokens.span.Span

In [ ]:
print(chunk.start_char, chunk.end_char)
print(chunk.start, chunk.end)



24 38
6 8


In [ ]:
doc.text[chunk.start_char:chunk.end_char] # character offset!

'the University'

In [ ]:
doc[chunk.start : chunk.end] # pozitii numarate ca tokeni, al cata-lea cuvant e?

the University

In [ ]:
list(doc.sents)

[Me and John Doe went to the University of Bucharest.,
 We were very happy about that.]

In [ ]:
token = doc[13]
print(type(token))
print(token)

<class 'spacy.tokens.token.Token'>
very


In [ ]:
token.dep_

'advmod'

### Part of speech tags
- https://universaldependencies.org/u/pos/
- tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
- full tag set for each model:

In [ ]:
token.pos_

'ADV'

In [ ]:
token.tag_

'RB'

In [ ]:
token.is_punct, token.is_stop

(False, True)

## Download data

In [ ]:
! rm -rf stylometry-federalist*
! wget https://programminghistorian.org/assets/introduction-to-stylometry-with-python/stylometry-federalist.zip

--2023-10-25 11:31:45--  https://programminghistorian.org/assets/introduction-to-stylometry-with-python/stylometry-federalist.zip
Resolving programminghistorian.org (programminghistorian.org)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to programminghistorian.org (programminghistorian.org)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 881447 (861K) [application/zip]
Saving to: ‘stylometry-federalist.zip’

stylometry-federali 100%[===================>] 860.79K  --.-KB/s    in 0.02s   

2023-10-25 11:31:46 (37.4 MB/s) - ‘stylometry-federalist.zip’ saved [881447/881447]



In [ ]:
! unzip stylometry-federalist.zip

Archive:  stylometry-federalist.zip
replace data/federalist_55.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! ls data

federalist_10.txt  federalist_27.txt  federalist_43.txt  federalist_5.txt   federalist_76.txt
federalist_11.txt  federalist_28.txt  federalist_44.txt  federalist_60.txt  federalist_77.txt
federalist_12.txt  federalist_29.txt  federalist_45.txt  federalist_61.txt  federalist_78.txt
federalist_13.txt  federalist_2.txt   federalist_46.txt  federalist_62.txt  federalist_79.txt
federalist_14.txt  federalist_30.txt  federalist_47.txt  federalist_63.txt  federalist_7.txt
federalist_15.txt  federalist_31.txt  federalist_48.txt  federalist_64.txt  federalist_80.txt
federalist_16.txt  federalist_32.txt  federalist_49.txt  federalist_65.txt  federalist_81.txt
federalist_17.txt  federalist_33.txt  federalist_4.txt	 federalist_66.txt  federalist_82.txt
federalist_18.txt  federalist_34.txt  federalist_50.txt  federalist_67.txt  federalist_83.txt
federalist_19.txt  federalist_35.txt  federalist_51.txt  federalist_68.txt  federalist_84.txt
federalist_1.txt   federalist_36.txt  federalist_52.txt  feder

In [ ]:
papers = {
    'Madison': [10, 14, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48],
    'Hamilton': [1, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 21, 22, 23, 24,
                 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 59, 60,
                 61, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77,
                 78, 79, 80, 81, 82, 83, 84, 85],
    'Jay': [2, 3, 4, 5],
    'Shared': [18, 19, 20],
    'Disputed': [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 62, 63],
    'TestCase': [64]
}

In [ ]:
## A function that compiles all of the text files associated with a single author into a single string
def read_files_into_string(filenames):
    strings = []
    for filename in filenames:
        with open(f'data/federalist_{filename}.txt', 'r') as f:
            strings.append(f.read())
    return '\n'.join(strings)

In [ ]:
federalist_by_author = {}
for author, files in papers.items():
    federalist_by_author[author] = read_files_into_string(files)

In [ ]:
federalist_by_author.keys()

dict_keys(['Madison', 'Hamilton', 'Jay', 'Shared', 'Disputed', 'TestCase'])

In [ ]:
federalist_by_author['Madison']

' 10\n\nThe Same Subject Continued (The Union as a Safeguard Against Domestic\nFaction and Insurrection)\n\nFrom the Daily Advertiser. Thursday, November 22, 1787.\n\nMADISON\n\nTo the People of the State of New York:\n\nAMONG the numerous advantages promised by a well constructed Union, none\ndeserves to be more accurately developed than its tendency to break and\ncontrol the violence of faction. The friend of popular governments never\nfinds himself so much alarmed for their character and fate, as when he\ncontemplates their propensity to this dangerous vice. He will not fail,\ntherefore, to set a due value on any plan which, without violating the\nprinciples to which he is attached, provides a proper cure for it.\nThe instability, injustice, and confusion introduced into the public\ncouncils, have, in truth, been the mortal diseases under which popular\ngovernments have everywhere perished; as they continue to be the\nfavorite and fruitful topics from which the adversaries to libert

In [ ]:
madison = NLP(federalist_by_author['Madison'])

In [ ]:
madison[:10]

 10

The Same Subject Continued (The Union

In [ ]:
for sentence in madison.sents:
    print(sentence)

Streaming output truncated to the last 5000 lines.
principal task of modern legislation, and involves the spirit of party
and faction in the necessary and ordinary operations of the government.


No man is allowed to be a judge in his own cause, because his interest
would certainly bias his judgment, and, not improbably, corrupt his
integrity.
With equal, nay with greater reason, a body of men are unfit
to be both judges and parties at the same time; yet what are many of the
most important acts of legislation, but so many judicial determinations,
not indeed concerning the rights of single persons, but concerning the
rights of large bodies of citizens?
And what are the different classes
of legislators but advocates and parties to the causes which they
determine?
Is a law proposed concerning private debts?
It is a question
to which the creditors are parties on one side and the debtors on the
other.
Justice ought to hold the balance between them.
Yet the parties
are, and must be, themselv

## Exercitiu

1. numarati cuvinte din fiecare document procesat cu spacy hint: `collections.Counter, sau scikit-learn.CountVectorizer`
2. selectati o lista cu cele mai frecvente 10 cuvinte
1. sau selectati liste de cuvinte cu anumite proprietati: conjuntii, prepozitii, adjective, token.is_stop, token.is_punct, ents, len(sentence), len(token)
3. plotati frecventele cuvintelor sau orice alte proprietati vreti sa extrageti din hamilton, madison si disputed
4. folositi [bar-chart plotly](https://plotly.com/python/bar-charts/) ca sa vedeti rezultatul intr-un mod interactiv

In [ ]:
from collections import Counter

for auth, texts in federalist_by_author.items():
    madison = NLP(texts)



## Exercitiu

1. aplicati modelul spaCy peste fiecare text in parte, nu peste intregul set de texte
1. construiti un vector de caracteristici pentru un text; caracteristicile pot fi: frecventele [cuvintelor functionale din articolul original](https://ptrckprry.com/course/ssd/reading/Most63.pdf), pg 25, lungimea medie a propozitiilor, numarul de entitati numite (named entities), lungimea medie a cuvintelor, nr de adjective, nr de semne de punctuatie ! Atentie, numerele trebuie normalizate astfel incat doua texte de lungimi diferite sa fie comparabile
1. aplicati functia de extractie a caracteristicilor pe toate textele
1. folosind o metoda de clustering agglomerativ, afisati dendrograma (arborele) corespunzator pentru datele voastre prin metoda Ward; [vezi aici tutorial](https://scikit-learn.org/stable/auto_examples/cluster/plot_agglomerative_dendrogram.html#sphx-glr-auto-examples-cluster-plot-agglomerative-dendrogram-py)
1. inlocuiti metoda cu o metrica [bazata pe string similrity (kernels)](https://www.jmlr.org/papers/volume2/lodhi02a/lodhi02a.pdf) sau pe RBF kernel; introduceti metrica de similaritate precalculata pentru a obtine linkage result; vezi documentatie din [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering)

In [ ]:
## A function that compiles all of the text files associated with a single author into a single string
def read_files_into_strings(filenames):
    strings = []
    for filename in filenames:
        with open(f'data/federalist_{filename}.txt', 'r') as f:
            strings.append(f.read())
    return strings

federalist_by_author = {}
for author, files in papers.items():
    federalist_by_author[author] = read_files_into_strings(files)

In [ ]:
federalist_by_author['Madison'][0]

' 10\n\nThe Same Subject Continued (The Union as a Safeguard Against Domestic\nFaction and Insurrection)\n\nFrom the Daily Advertiser. Thursday, November 22, 1787.\n\nMADISON\n\nTo the People of the State of New York:\n\nAMONG the numerous advantages promised by a well constructed Union, none\ndeserves to be more accurately developed than its tendency to break and\ncontrol the violence of faction. The friend of popular governments never\nfinds himself so much alarmed for their character and fate, as when he\ncontemplates their propensity to this dangerous vice. He will not fail,\ntherefore, to set a due value on any plan which, without violating the\nprinciples to which he is attached, provides a proper cure for it.\nThe instability, injustice, and confusion introduced into the public\ncouncils, have, in truth, been the mortal diseases under which popular\ngovernments have everywhere perished; as they continue to be the\nfavorite and fruitful topics from which the adversaries to libert

In [ ]:
# preferam batching in loc de a aplicat NLP() pe fiecare text
for document in NLP.pipe(federalist_by_author['Madison'], batch_size=10):
  print(len(document), )

3662
2613
3287
4030
3216
3710
4304
3392
4245
3531
2596
3164
3400
2327


In [ ]:
from collections import defaultdict

autor2spacy = defaultdict(list)

for autor, texte in federalist_by_author.items():
  for document in NLP.pipe(texte, batch_size=10):
    autor2spacy[autor].append(document)



In [ ]:
autor2spacy['Madison'][0][:100]

 10

The Same Subject Continued (The Union as a Safeguard Against Domestic
Faction and Insurrection)

From the Daily Advertiser. Thursday, November 22, 1787.

MADISON

To the People of the State of New York:

AMONG the numerous advantages promised by a well constructed Union, none
deserves to be more accurately developed than its tendency to break and
control the violence of faction. The friend of popular governments never
finds himself so much alarmed for their character and fate, as when he

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
from spacy.tokens.doc import Doc
import numpy as np
from collections import Counter

IMPORTANT_WORDS = set(["upon", "upon", "also", "an", "by", "of", "there", "this", "to", "although", "both"])
# vrem sa impunem o ordine a caracteristicilor
# astfel incat pe pozitia 3 sa avem intotdeauna codificat acelasi feature
CUVINTE_IMPORTANTE_UNICE_SORTATE = sorted(list(IMPORTANT_WORDS))
print(CUVINTE_IMPORTANTE_UNICE_SORTATE)

def featurize(document: Doc) -> np.array:
  contor = Counter()
  num_alpha = 0
  for token in document:
    if token.is_alpha:
      num_alpha += 1
    if token.text.lower() in IMPORTANT_WORDS:
      contor[token.text.lower()] += 1
  rezultat = np.zeros(len(IMPORTANT_WORDS))
  for idx, cuvant in enumerate(CUVINTE_IMPORTANTE_UNICE_SORTATE):
    rezultat[idx] = contor[cuvant]
  return rezultat / num_alpha



['also', 'although', 'an', 'both', 'by', 'of', 'there', 'this', 'to', 'upon']


In [ ]:
print(featurize(autor2spacy['Madison'][0]))
print(featurize(autor2spacy['Hamilton'][0]))

[0.         0.         0.00462046 0.00165017 0.01287129 0.05181518
 0.0019802  0.00363036 0.0330033  0.        ]
[0.         0.         0.00675261 0.         0.00859423 0.0650706
 0.00122775 0.00859423 0.0441989  0.00368324]


In [ ]:
from typing import List
def documente2vectori(documente: List[Doc]) -> np.array:
  """Can be improved. :)
  """
  X = []
  for document in autor2spacy['Madison']:
    X.append(featurize(document))
  return np.array(X)


In [ ]:
X = []
labels = []
for autor in ["Madison", "Hamilton", "Disputed"]:
  for document in autor2spacy[autor]:
    X.append(featurize(document))
    labels.append(autor)

X = np.array(X)


In [ ]:
X.shape

(77, 10)

In [ ]:
# celulele de mai jost sunt hacked, vezi linkagefun
# folsiti mai bine AgglomerativeClustering din scikit-learn
from scipy.cluster.hierarchy import linkage
# functia linkage din scipy
rezultat_clustering = linkage(X, 'ward', metric='euclidean')

In [ ]:
import plotly.figure_factory as ff

fig = ff.create_dendrogram(X, linkagefun=lambda dummy:rezultat_clustering, orientation='left', labels=labels)
fig.update_layout(width=800, height=500)
fig.show()